In [5]:
!pip3 install pandas
!pip3 install sklearn
!pip3 install tqdm

  Using cached tqdm-4.61.0-py2.py3-none-any.whl (75 kB)


In [6]:
#coding=utf-8
import csv
import os
import numpy as np
import pandas as pd
import jieba
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from sklearn.feature_selection import mutual_info_classif
from tqdm import tqdm
from collections import Counter
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import KNeighborsClassifier

In [8]:
def stopwordslist():
    stopwords = [line.strip() for line in open('null.txt', encoding='UTF-8').readlines()]
    return stopwords

def seg_depart(sentence):
    # 对文档中的每一行进行中文分词
    # print("正在分词")
    sentence_depart = jieba.cut(str(sentence).strip())
    # 创建一个停用词列表
    stopwords = stopwordslist()
    # 输出结果为outstr
    outstr = ''
    # 去停用词
    for word in sentence_depart:
        if word not in stopwords:
            if word != '\t':
                outstr += word
                outstr += " "
    return outstr
def data_get():
    data_0 = pd.read_csv("体育.csv")
    data_1 = pd.read_csv("彩票.csv")
    data_2 = pd.read_csv("房产.csv")
    data_3 = pd.read_csv("时政.csv")
    data_4 = pd.read_csv("社会.csv")
    data_5 = pd.read_csv("股票.csv")
    data_6 = pd.read_csv("财经.csv")
    data = pd.concat([data_0, data_1, data_2, data_3, data_4, data_5, data_6]).reset_index(drop = True)
    return data

def train():
    x_train,x_test,y_train,y_test = train_test_split(seg_depart(data['text']), data.class_name, test_size=0.2, random_state=2020)

    pipe = Pipeline([('vect', CountVectorizer()),
                     ('tfidf', TfidfTransformer(binary=True)),
                     ('model', LogisticRegression())])

    model = pipe.fit(x_train, y_train)
    prediction = model.predict(x_test)
    print("accuracy: {}%".format(round(accuracy_score(y_test, prediction)*100,2)))

In [10]:
data = data_get()

In [11]:
newData = pd.Series(len(data.text))

In [12]:
for i in tqdm(range(len(data.text))):
        newData[i] = seg_depart(data.text[i])

  0%|          | 0/126920 [00:00<?, ?it/s]Building prefix dict from the default dictionary ...
Loading model from cache /var/folders/yj/fg65d18x5r51k2d5h2_ql1w40000gn/T/jieba.cache
Loading model cost 0.754 seconds.
Prefix dict has been built successfully.
100%|██████████| 126920/126920 [53:39<00:00, 39.42it/s] 


In [13]:
 x_train, x_test, y_train, y_test = train_test_split(newData, data.class_name, test_size=0.2,
                                                        random_state=2020)

In [36]:
from sklearn.svm import LinearSVC
pipe = Pipeline([('vect', CountVectorizer()),
                     ('tfidf',TfidfTransformer()),
                     ('model', LinearSVC())])

model = pipe.fit(x_train, y_train)

In [37]:
prediction = model.predict(x_test)
print("accuracy: {}%".format(round(accuracy_score(y_test, prediction) * 100, 2)))
print(confusion_matrix(y_test, prediction))
print(classification_report(y_test, prediction))

accuracy: 96.14%
[[3707   15    1    6    9    2    1]
 [  14 1434    0    2   28    0    1]
 [   1    0 3951   36   55   24    6]
 [  14    2   20 4318   74   78   29]
 [   4   13   17   62 3963    3   20]
 [   3    0    8   91    9 4335   74]
 [   2    0   28   42   49  137 2696]]
              precision    recall  f1-score   support

          体育       0.99      0.99      0.99      3741
          彩票       0.98      0.97      0.97      1479
          房产       0.98      0.97      0.98      4073
          时政       0.95      0.95      0.95      4535
          社会       0.95      0.97      0.96      4082
          股票       0.95      0.96      0.95      4520
          财经       0.95      0.91      0.93      2954

    accuracy                           0.96     25384
   macro avg       0.96      0.96      0.96     25384
weighted avg       0.96      0.96      0.96     25384

